# PART 1 - Analysis (Excel)

## Import necessary libraries

In [228]:
import pandas as pd
import matplotlib.pyplot as plt

## Import Data from Excel file

In [229]:
df = pd.read_excel("../data/marketing_raw_data.xlsx")

df

,Date,Time,Order ID,Customer ID,Coupon Code,Country,Total
0,2020-10-01,11:52:28,199652320,69987.0,NaN,CA,51.55
1,2020-10-01,11:54:52,199652339,56440.0,NaN,US,50.55
2,2020-10-01,11:57:14,199652338,77646.0,NaN,US,55.07
3,2020-10-01,11:59:26,199652344,6041.0,NaN,US,91.20
4,2020-10-01,12:01:58,199625188-1,43125.0,NaN,US,59.19
...,...,...,...,...,...,...,...
64279,2020-11-30,23:34:19,199780389,81869.0,NaN,US,55.56
64280,2020-11-30,23:40:13,199780382,64591.0,5AFG,US,9.66
64281,2020-11-30,23:40:32,199780383,27503.0,NaN,US,90.55
64282,2020-11-30,23:43:24,199780385,95286.0,NaN,US,52.51


## 1. Check the database in the attached Excel file for errors. If error found, write down what changed and why

In [230]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64284 entries, 0 to 64283
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         64284 non-null  datetime64[ns]
 1   Time         64284 non-null  object        
 2   Order ID     64283 non-null  object        
 3   Customer ID  64283 non-null  float64       
 4   Coupon Code  6244 non-null   object        
 5   Country      64284 non-null  object        
 6   Total        64284 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 3.4+ MB


### Error: NaN value for Order ID and for Client ID
From dataframe info we can see, that there is one NaN value for Order ID and for Client ID. I've found out this values are from the one record. First option could be to add sample ids. In this case, I'll delete this record, since it won't make a big difference in analysis and this record could be some system bag.

In [231]:
df = df.dropna(subset="Order ID")

In [232]:
df.describe()

,Date,Customer ID,Total
count,64283,64283.000000,64283.000000
mean,2020-11-02 10:35:48.362708736,48460.790116,57.559169
min,2019-10-01 00:00:00,1.000000,-52.950000
25%,2020-10-20 00:00:00,24315.000000,52.060000
50%,2020-11-05 00:00:00,48688.000000,55.050000
75%,2020-11-16 00:00:00,72487.500000,59.000000
max,2020-11-30 00:00:00,96444.000000,1170.000000
std,NaN,27810.631969,22.607637


In [233]:
df[df["Total"] < 0]

,Date,Time,Order ID,Customer ID,Coupon Code,Country,Total
30947,2020-11-04,17:02:47,199862386,31810.0,NaN,US,-52.95


### Error: negative value in Total column

There is negative value in Total column. I suggest, it was a mistake and the value should be positive.

In [234]:
df.loc[:, "Total"] = df["Total"].abs()

df.describe()

,Date,Customer ID,Total
count,64283,64283.000000,64283.000000
mean,2020-11-02 10:35:48.362708736,48460.790116,57.560817
min,2019-10-01 00:00:00,1.000000,0.000000
25%,2020-10-20 00:00:00,24315.000000,52.060000
50%,2020-11-05 00:00:00,48688.000000,55.050000
75%,2020-11-16 00:00:00,72487.500000,59.000000
max,2020-11-30 00:00:00,96444.000000,1170.000000
std,NaN,27810.631969,22.603442


### Error: 25th, 75th percentiles are around save values, but max is much higher

Median and 25th, 75th percentiles are around 50, but max Total value is 1170. It could be just outlier, but almost same order cost for the biggest part of data also doesn't seem right.

In [235]:
max_value = df["Total"].max()
bins = list(range(0, int(max_value) + 100, 100))
counts_df = pd.DataFrame(
    {
        "Order ID": df["Order ID"],
        "Price Range": pd.cut(df["Total"], bins=bins, right=False),
    }
)

counts_df = (
    counts_df.groupby("Price Range", observed=False)["Order ID"].count().reset_index()
)

counts_df.columns = ["Price Range", "Number of Orders"]

counts_df

,Price Range,Number of Orders
0,"[0, 100)",63513
1,"[100, 200)",721
2,"[200, 300)",37
3,"[300, 400)",0
4,"[400, 500)",0
5,"[500, 600)",9
6,"[600, 700)",2
7,"[700, 800)",0
8,"[800, 900)",0
9,"[900, 1000)",0


### Error: outliers

We can see now that most Total values are in 0-100 range. Much less are in 100-200 and 200-300 range. I suggest all values from 500, or even from 200 could be outliers, that will affect our analysis. In this case standart deviation or quartile outliers defining will give us only 0-100 values. Later in more detailed analysis I'll remove values that are higher than 500 and suggest that Total in 100-300 range is ok and won't affect our analysis in a bad way. But for now I'll just remove value in 1100-1200, which could indicate an error in our data.

In [236]:
df = df[df["Total"] < 1100]

In [237]:
df[df["Total"] == 0]

,Date,Time,Order ID,Customer ID,Coupon Code,Country,Total
27,2020-10-01,12:32:02,199652422,80.0,5AWS,US,0.0
206,2020-10-01,16:41:49,199653945,81.0,USTF,US,0.0
618,2020-10-02,08:19:05,199654435,37375.0,5AWS,CA,0.0
671,2020-10-02,09:08:22,199654613,42031.0,5AWS,US,0.0
690,2020-10-02,09:21:46,199654666,62696.0,35WS,US,0.0
...,...,...,...,...,...,...,...
64141,2020-11-30,18:30:00,199787749,20530.0,5ARK,US,0.0
64151,2020-11-30,18:46:30,199787787,7216.0,5ARQ,US,0.0
64164,2020-11-30,19:06:10,199787033,32214.0,5AR.,US,0.0
64204,2020-11-30,20:29:22,199780133,66106.0,5AR.,US,0.0


In [238]:
df.sort_values("Date").head()

,Date,Time,Order ID,Customer ID,Coupon Code,Country,Total
470,2019-10-01,03:46:19,199653051,4656.0,NaN,US,90.95
383,2019-10-01,21:10:46,199653651,31325.0,NaN,US,51.00
0,2020-10-01,11:52:28,199652320,69987.0,NaN,CA,51.55
300,2020-10-01,19:02:07,199653380,8863.0,NaN,CA,51.55
299,2020-10-01,19:01:16,199653387,12186.0,NaN,US,51.65


### Error: dates for 2019 year
I suggest, it could be a system error or a data entry mistake and the actual year is 2020 for these dates since they have accurate month and day. I'll change years of these dates to 2022.

In [239]:
df["Date"] = df["Date"].apply(
    lambda date: date.replace(year=2020) if date.year == 2019 else date
)

df

,Date,Time,Order ID,Customer ID,Coupon Code,Country,Total
0,2020-10-01,11:52:28,199652320,69987.0,NaN,CA,51.55
1,2020-10-01,11:54:52,199652339,56440.0,NaN,US,50.55
2,2020-10-01,11:57:14,199652338,77646.0,NaN,US,55.07
3,2020-10-01,11:59:26,199652344,6041.0,NaN,US,91.20
4,2020-10-01,12:01:58,199625188-1,43125.0,NaN,US,59.19
...,...,...,...,...,...,...,...
64279,2020-11-30,23:34:19,199780389,81869.0,NaN,US,55.56
64280,2020-11-30,23:40:13,199780382,64591.0,5AFG,US,9.66
64281,2020-11-30,23:40:32,199780383,27503.0,NaN,US,90.55
64282,2020-11-30,23:43:24,199780385,95286.0,NaN,US,52.51


In [240]:
df[(df["Total"] == 0) & (df["Coupon Code"].isnull())]

,Date,Time,Order ID,Customer ID,Coupon Code,Country,Total


### Error (clarification): zero totals

As we can see, there is no orders with zero total and no used coupon. That could mean that we have some coupons that cover all order price. I'll consider it like our business logic allows coupons to cover full order price.

In [241]:
# check if country names are consistent

df["Country"].unique()

array(['CA', 'US'], dtype=object)

In [242]:
df["Order ID"].is_unique

False

In [243]:
df[df["Order ID"].duplicated(keep=False)]

,Date,Time,Order ID,Customer ID,Coupon Code,Country,Total
8000,2020-10-11,07:17:47,199679736,86794.0,NaN,CA,56.55
15765,2020-10-20,07:48:36,199890016,68269.0,NaN,CA,56.55
32708,2020-11-05,19:41:20,199679736,86794.0,NaN,CA,56.55
32709,2020-11-05,19:41:20,199890016,68269.0,NaN,CA,56.55
36759,2020-11-08,11:27:45,199872707,36231.0,NaN,US,51.00
45777,2020-11-14,19:38:52,199713829,60591.0,NaN,CA,56.55
54721,2020-11-20,11:27:45,199872707,36231.0,NaN,US,51.00
56928,2020-11-21,19:38:52,199713829,60591.0,NaN,CA,56.55


### Error: Order ID duplicates

Order ID duplicates, which is not appropriate, since ids should be unique. This could suggest a potential issue, such as orders being recorded multiple times due to a system error or a data entry mistake. In this case, I'll keep first order id occurance, since all data, besides date, duplicates.

In [244]:
df = df.drop_duplicates("Order ID", keep="first")

df[df["Order ID"].duplicated(keep=False)]

,Date,Time,Order ID,Customer ID,Coupon Code,Country,Total


In [245]:
df[df["Customer ID"].apply(lambda x: x % 1 != 0)]

,Date,Time,Order ID,Customer ID,Coupon Code,Country,Total


### Error (data type correction): Customer ID values are floats

By the dataframe info and the last check, we can see that all Customer ID values are floats with only zeros after a coma. That means, we can change type for this column to int.

In [246]:
# change dataframe data types to move efficient and memoryusage-friendly

df = df.astype(
    {
        "Customer ID": "int",
        "Order ID": "str",
        "Coupon Code": "category",
        "Country": "category",
        "Total": "float32",
    }
)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64278 entries, 0 to 64283
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         64278 non-null  datetime64[ns]
 1   Time         64278 non-null  object        
 2   Order ID     64278 non-null  object        
 3   Customer ID  64278 non-null  int32         
 4   Coupon Code  6244 non-null   category      
 5   Country      64278 non-null  category      
 6   Total        64278 non-null  float32       
dtypes: category(2), datetime64[ns](1), float32(1), int32(1), object(2)
memory usage: 2.6+ MB


In [247]:
df.head()

,Date,Time,Order ID,Customer ID,Coupon Code,Country,Total
0,2020-10-01,11:52:28,199652320,69987,NaN,CA,51.549999
1,2020-10-01,11:54:52,199652339,56440,NaN,US,50.549999
2,2020-10-01,11:57:14,199652338,77646,NaN,US,55.070000
3,2020-10-01,11:59:26,199652344,6041,NaN,US,91.199997
4,2020-10-01,12:01:58,199625188-1,43125,NaN,US,59.189999


In [248]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 64278 entries, 0 to 64283
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         64278 non-null  datetime64[ns]
 1   Time         64278 non-null  object        
 2   Order ID     64278 non-null  object        
 3   Customer ID  64278 non-null  int32         
 4   Coupon Code  6244 non-null   category      
 5   Country      64278 non-null  category      
 6   Total        64278 non-null  float32       
dtypes: category(2), datetime64[ns](1), float32(1), int32(1), object(2)
memory usage: 2.6+ MB


## 2. Add a column for day of the week

In [249]:
df["Week"] = df["Date"].dt.isocalendar().week
df["Week"] = df["Date"].dt.to_period("W").apply(lambda week: week.start_time)

df.head()

,Date,Time,Order ID,Customer ID,Coupon Code,Country,Total,Week
0,2020-10-01,11:52:28,199652320,69987,NaN,CA,51.549999,2020-09-28
1,2020-10-01,11:54:52,199652339,56440,NaN,US,50.549999,2020-09-28
2,2020-10-01,11:57:14,199652338,77646,NaN,US,55.070000,2020-09-28
3,2020-10-01,11:59:26,199652344,6041,NaN,US,91.199997,2020-09-28
4,2020-10-01,12:01:58,199625188-1,43125,NaN,US,59.189999,2020-09-28


## 3. Calculate the exchange rate of all Orders

Create Is Exchange column to indicate exchange orders

In [250]:
df["Is Exchange"] = df["Order ID"].str.contains("-").astype(int)

df.head()

,Date,Time,Order ID,Customer ID,Coupon Code,Country,Total,Week,Is Exchange
0,2020-10-01,11:52:28,199652320,69987,NaN,CA,51.549999,2020-09-28,0
1,2020-10-01,11:54:52,199652339,56440,NaN,US,50.549999,2020-09-28,0
2,2020-10-01,11:57:14,199652338,77646,NaN,US,55.070000,2020-09-28,0
3,2020-10-01,11:59:26,199652344,6041,NaN,US,91.199997,2020-09-28,0
4,2020-10-01,12:01:58,199625188-1,43125,NaN,US,59.189999,2020-09-28,1


In [251]:
df[df["Is Exchange"] == 1]["Coupon Code"].unique()

[NaN, 'WABl', '3AWS', '18AW', 'INF1', '35WS']
Categories (203, object): [12.97, '13AW', '145W', '15.A', ..., 'rABv', 'trAB', 'ustf', 'wABl']

### Error: coupons for exchange orders

It could possibly be an error that we have coupons for exchange orders. Since I don't have enough information, I'll suggest that we have some coupons for orders delivery, for example, and will keep this data like it is.

Calculate the exchange rate

In [252]:
number_of_exchanges = df["Is Exchange"].sum()

total_orders = df.shape[0]

exchange_rate = (number_of_exchanges / total_orders) * 100

exchange_rate

5.020380223404587

### Exchange rate of all Orders is equal to 5.02%

## 4. What is the repurchase rate for October Customers?

Get purchases for unique October Customers

In [253]:
oct_df = df[df["Date"].dt.month == 10]

oct_customers = oct_df["Customer ID"].unique()

oct_purchases_df = df[df["Customer ID"].isin(oct_customers)]

oct_purchases_df.head(3)

,Date,Time,Order ID,Customer ID,Coupon Code,Country,Total,Week,Is Exchange
0,2020-10-01,11:52:28,199652320,69987,NaN,CA,51.549999,2020-09-28,0
1,2020-10-01,11:54:52,199652339,56440,NaN,US,50.549999,2020-09-28,0
2,2020-10-01,11:57:14,199652338,77646,NaN,US,55.070000,2020-09-28,0


Calculate repurchase rate for October Customers

In [254]:
oct_repurchase_df = oct_purchases_df["Customer ID"].value_counts()
oct_repurchasers = oct_repurchase_df[
    oct_repurchase_df > 1
].index  # Customers who purchased more than once

oct_customers_repurchase_rate = len(oct_repurchasers) / len(oct_customers) * 100

oct_customers_repurchase_rate

11.21231925496283

### Repurchase rate for October Customers is equal to 11.21%

## 5. Who is using more coupon codes, US or Canadian customers?

In [255]:
coupon_df = df[df["Coupon Code"].notnull()]

coupon_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6244 entries, 19 to 64283
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         6244 non-null   datetime64[ns]
 1   Time         6244 non-null   object        
 2   Order ID     6244 non-null   object        
 3   Customer ID  6244 non-null   int32         
 4   Coupon Code  6244 non-null   category      
 5   Country      6244 non-null   category      
 6   Total        6244 non-null   float32       
 7   Week         6244 non-null   datetime64[ns]
 8   Is Exchange  6244 non-null   int32         
dtypes: category(2), datetime64[ns](2), float32(1), int32(2), object(2)
memory usage: 345.2+ KB


In [256]:
country_coupon_usage = coupon_df["Country"].value_counts()

country_coupon_usage

Country
US    5603
CA     641
Name: count, dtype: int64

US customers used 5603 coupons.
Canadian customers used 641 coupons.

### US customers use more coupon codes then Canadian.

## 6. Possible Explanations:
1. **Location**. IL Makiage company is located in the New York, so the delivery will be faster and cheeper for the citizens of US due to various customs restrictions. The delivery could be also much faster. That's why US customers are tended to make more purchases and more likely to respond to our marketing.
2. **Time on the market**. For the reason of company location, I suggest company was longer on a domestic market. That's why US customers can have more confidence in the company.
3. **Strategic Focus**. A US company might prioritize its domestic market in terms of marketing investment and strategic focus. This could include more frequent or more appealing promotions, including coupon codes, targeted at US customers. At the same time, the approach in Canada might be less aggressive or developed, possibly due to fewer resources allocated to understanding and capturing market nuances there.
4. **Local Market Familiarity**. Being a US-based company could mean that the company has deeper insights and a more established presence in the US market compared to Canada. This familiarity can lead to more effective marketing strategies, including the use of coupon codes, which are better aligned with US consumer behaviors and preferences.
5. **Cultural Differences**: Purchase of makeup products may be related to the extent to which Canadians and US citizens are inclined to use them. Probably US residents are more likely to buy such products and that there is a greater demand for them in Canada.

## 7. Choose 2 possible reasons from the ones mentioned above. What data will be needed to examine each hypothesis

### 1. Strategic Focus
**Data Needed:**
1. Marketing Budget Allocation: Examine how the marketing budget is distributed between the US and Canadian markets. This includes funds allocated to promotions, advertising, and coupon campaigns.
2. Campaign Performance Data: Collect data on the performance of marketing campaigns in both countries. Metrics to consider include engagement rates, conversion rates, and ROI from campaigns that involved coupon codes.
3. Internal Strategy Documents: Review strategic planning documents that outline market priorities and resource allocation. This can provide insights into whether there is a deliberate focus on the US market over Canada.

### 2. Local Market Familiarity
**Data Needed:**
1. Sales and Customer Data: Review sales data and customer demographics/psychographics for insights into market penetration and customer profiles in both countries.
2. Market Research Reports: Obtain comprehensive market research reports for both the US and Canadian markets. These should provide insights into consumer behavior, preferences, and attitudes towards promotions such as coupons.
3. Customer Feedback: Analyze customer feedback and survey data from both markets. This could reveal how well the company understands and meets the needs and expectations of customers in each region.

## 8. Review data

In [ ]:
df.to_excel("../data/marketing_cleaned_data.xlsx", sheet_name="Data", index=False)

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df.groupby("Week")["Order ID"].count(), label="Total Orders")
plt.plot(
    df[~df["Coupon Code"].isnull()].groupby("Week")["Coupon Code"].count(),
    label="Coupons Used",
)
plt.title("Orders amount by Week")
plt.xlabel("Date")
plt.ylabel("Orders")
plt.legend()
plt.show()

**Here we can see that the number of coupons used has almost no effect on the change in the number of orders**

In [ ]:
purchases_df = df[df["Is Exchange"] == 0]
exchange_df = df[df["Is Exchange"] == 1]

purchases_df.head(3)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(df.groupby("Week")["Total"].sum(), label="Total Orders")
plt.plot(purchases_df.groupby("Week")["Total"].sum(), label="Regular Purchases")
plt.plot(exchange_df.groupby("Week")["Total"].sum(), label="Exchanges")
plt.xlabel("Date")
plt.ylabel("Orders Cost, $")
plt.legend()
plt.show()

**Here we can see that amount of Exchanges is not really sagnificant and reacts quite a bit to the increase in the total amount of orders by period**, which could show that they are not really effective in terms for maximizing profits in a short period of time.

## 9. More insights

1. **Revenue Analysis with Coupon Usage**. Understanding the revenue generated from purchases made with coupons can provide critical insights into how effective your coupon strategy is at driving sales. By analyzing this data, we'll be able to identify trends such as which coupons are most effective, at what times coupon usage leads to higher sales, and how coupons impact the average order value. This information would help in optimizing the issuance of coupons to maximize profits.
2. **User Behavior Analysis** Gathering additional user data to analyze the demographics and behavior patterns of users who use coupons can substantially enhance your marketing strategies. By understanding which segments of a user base are more likely to use coupons, we'll be able tailor marketing efforts to target these groups more effectively. This could involve personalizing offers based on user preferences, shopping behavior, or even geographical data (both for different counntries and for different regiouns of the same country).

4. **Last but not least :)** In the real case, I'd communicate with team about all errors with dataset to decide what records I should keep, fix or remove and to prevent same issues.